In [1]:
import os
from pathlib import Path

import torch
from IPython.display import Markdown

import config as cfg

In [10]:
OUTPUT_PATH = "./Deeplabv3P_resnet50_batch16"
DATA_TYPE = "FP32"
BASE_MODEL_NAME = OUTPUT_PATH + "/" + DATA_TYPE

IMAGE_WIDTH = 1600
IMAGE_HEIGHT = 800
DEVICE='cuda'

In [11]:
if not os.path.exists(OUTPUT_PATH):
    os.mkdir(OUTPUT_PATH)

# Paths where PyTorch, ONNX and OpenVINO IR models will be stored.
model_path = Path(BASE_MODEL_NAME).with_suffix(".pth")
onnx_path = model_path.with_suffix(".onnx")
ir_path = model_path.with_suffix(".xml")

In [12]:
def export_onnx():
    dummy_input = torch.randn(1, 3, IMAGE_HEIGHT, IMAGE_WIDTH).to(DEVICE)
    model = torch.load('./Models/Deeplabv3P_resnet50_batch16.pth')
    model.eval()
    torch.onnx.export(model.module, dummy_input, onnx_path, verbose=True, opset_version=11)    

In [13]:
def create_mo_command():
    mo_cmd = f"""mo
            --input_model "{onnx_path}"
            --input_shape "[1, 3, {IMAGE_HEIGHT}, {IMAGE_WIDTH}]"
            --data_type FP16
            --output_dir "{OUTPUT_PATH}"
            """
    mo_cmd = " ".join(mo_cmd.split())
    print("Model Optimizer command to convert the ONNX model to OpenVINO:")
    display(Markdown(f"`{mo_cmd}`"))
    return mo_cmd

In [14]:
def convert_onnx_to_openvino(mo_cmd):
    if not ir_path.exists():
        print("Exporting ONNX model to IR... This may take a few minutes.")
        mo_result = %sx $mo_cmd
        print("\n".join(mo_result))
        print("DONE")
    else:
        print(f"IR model {ir_path} already exists.")    

In [15]:
export_onnx()

graph(%input.1 : Float(1, 3, 800, 1600, strides=[3840000, 1280000, 1600, 1], requires_grad=0, device=cuda:0),
      %decoder.aspp.0.convs.1.0.0.weight : Float(2048, 1, 3, 3, strides=[9, 9, 3, 1], requires_grad=1, device=cuda:0),
      %decoder.aspp.0.convs.2.0.0.weight : Float(2048, 1, 3, 3, strides=[9, 9, 3, 1], requires_grad=1, device=cuda:0),
      %decoder.aspp.0.convs.3.0.0.weight : Float(2048, 1, 3, 3, strides=[9, 9, 3, 1], requires_grad=1, device=cuda:0),
      %decoder.aspp.1.0.weight : Float(256, 1, 3, 3, strides=[9, 9, 3, 1], requires_grad=1, device=cuda:0),
      %decoder.block2.0.0.weight : Float(304, 1, 3, 3, strides=[9, 9, 3, 1], requires_grad=1, device=cuda:0),
      %segmentation_head.0.weight : Float(1, 256, 1, 1, strides=[256, 1, 1, 1], requires_grad=1, device=cuda:0),
      %segmentation_head.0.bias : Float(1, strides=[1], requires_grad=1, device=cuda:0),
      %619 : Float(64, 3, 7, 7, strides=[147, 49, 7, 1], requires_grad=0, device=cuda:0),
      %620 : Float(64, 

In [16]:
mo_cmd = create_mo_command()

Model Optimizer command to convert the ONNX model to OpenVINO:


`mo --input_model "Deeplabv3P_resnet50_batch16/FP32.onnx" --input_shape "[1, 3, 800, 1600]" --data_type FP16 --output_dir "./Deeplabv3P_resnet50_batch16"`

In [17]:
convert_onnx_to_openvino(mo_cmd)

Exporting ONNX model to IR... This may take a few minutes.
Model Optimizer arguments:
Common parameters:
	- Path to the Input Model: 	/home/shung/STUDIO/Intel/TEAM_1114_STAS_II/Deeplabv3P_resnet50_batch16/FP32.onnx
	- Path for generated IR: 	/home/shung/STUDIO/Intel/TEAM_1114_STAS_II/./Deeplabv3P_resnet50_batch16
	- IR output name: 	FP32
	- Log level: 	ERROR
	- Batch: 	Not specified, inherited from the model
	- Input layers: 	Not specified, inherited from the model
	- Output layers: 	Not specified, inherited from the model
	- Input shapes: 	[1, 3, 800, 1600]
	- Source layout: 	Not specified
	- Target layout: 	Not specified
	- Layout: 	Not specified
	- Mean values: 	Not specified
	- Scale values: 	Not specified
	- Scale factor: 	Not specified
	- Precision of IR: 	FP16
	- Enable fusing: 	True
	- User transformations: 	Not specified
	- Reverse input channels: 	False
	- Enable IR generation for fixed input shape: 	False
	- Use the transformations config file: 	None
Advanced parameters:
	- 